# Occlusion

### What it this?

This script takes images from a given directory and returns their occluded version.

### How does it work?

Very good question. First check the following requirements:
1. Make sure this script is located in a directory of which the parent directory contains a folder named `images` and where all the images you wish to work on are located.
2. Make sure the input images are 4-channel, with the background gray & transparent.

Then follow these simple steps for a smooth experience:
1. Make sure you select the options you wish for in the preamble of this script. In particular, set the following to `True`/`False` to determine the output(s) you will get:

- **deletion**: if set to `True`, images will be occluded with blobs the same color as the background;
- **partialViewing**: if set to `True`, images will be occluded fully with openings revealing parts of the objects;
- **blob**: if set to `True`, images will be occluded with blobs of a color different than the background.

2. Select the color you wish for the partial viewing and blob manipulations. Default is black.
3. Select your preference between many small occluders and few large occluders (choose only one).

## Preamble

In [1]:
import cv2 as cv
import numpy as np
import glob
import os
import time
from numpy.random import randint
print("All libraries are loaded.")

All libraries are loaded.


In [2]:
np.random.seed()

A note on the naming: in variable names, I always use *high* and *low* with reference to high and low **levels of occlusion**. Consequently:
- In the _high_ condition, only a small portion of the object is visible.
- In the _low_ condition, a large portion of the object remains visible.

In [3]:
# decide if you prefer many small occluders OR few large occluders
manySmall = False
fewLarge = True

if manySmall:
    sizeOccluder = [20, 90]
    nOccluders_low = 15
    nOccluders_high = 50
    
elif fewLarge:
    sizeOccluder = [70, 300]
    nOccluders_low = 5
    nOccluders_high = 5

# decide on the portion of object **still visible** in high and low **levels of occlusion**
portionHigh = 30
portionLow = 70

In [4]:
# decide on the colors to use, black by default
colPartialViewing = 0 # color of the occluding window (grayscale)
colBlob = (0,0,0) # color of the occluding blobs

In [5]:
# decide on the manipulations
doDeletion = True
doPartialViewing = True
doBlob = True

doManipulations = (doDeletion, doPartialViewing, doBlob)



### Set the input path

In [6]:
# give the input path
parentPath = os.pardir
inputPath = parentPath + '/images/'

imagePath = glob.glob(inputPath + "*") # take in all the files in there
imagePath.sort() # make sure the files are sorted

# count the number of images to process
nbrIm = len(imagePath)

print('The input path is ', inputPath, ' and contains ', nbrIm, ' images.')

The input path is  ../images/  and contains  80  images.


### Set the output path

In [7]:
# name the output paths
outputPath = parentPath + r'/output/occlusionOuput/'
outputPathDeletion = outputPath + 'deletion/'
outputPathPartialViewing = outputPath + 'partialViewing/'
outputPathBlob = outputPath + 'blob/'

listOutputPaths = [outputPathDeletion, outputPathPartialViewing, outputPathBlob]

# create the necessary output folders
# manipulations = (deletion, partialViewing, blob)
idx = 0

for manipulation in doManipulations:
    if manipulation:
        if not os.path.exists(listOutputPaths[idx]):
            os.makedirs(listOutputPaths[idx])
            idx +=1

## Start occluding

**Deletion & Blobs**: a word on the technique

The way deletion and blobs work so that they only cover a given portion of the object itself (and only of the object, not of the whole image) is the following:
1. When importing the image, an object mask is extracted
2. When patches of the image are deleted/blobs are added, an occluder mask is created
3. The intersection of both masks is measured (in pixel size)
4. The proportion of that intersection to the total object size is calculated. From there:
    - If it falls within the desired boundaries (e.g. 50% of object occluded) the operation is stopped and the image saved
    - It if doesn't, one occluder is added/removed depending on the sign of the difference, and the operation is run again.

In [8]:
# find the BGR color of the background from the upper left-most pixel
colBckgrnd = ((cv.imread(imagePath[0], -1))[0,0,0:3])
colBckgrnd = int(colBckgrnd[0]), int(colBckgrnd[1]), int(colBckgrnd[2])
print('Color background is', colBckgrnd)

Color background is (107, 107, 107)


The following three cells each run one of the occluding manipulations. Only the manipulations that have been set to `True` in the preamble will be applied.

In [9]:
# deletion

if doManipulations[0]:
    for file in imagePath:
        
        # read the image
        img = cv.imread(file, -1)
        
        # extract image name
        basename = os.path.basename(file)
        name = os.path.splitext(basename)[0]

        # creating object masks & measuring pixel size
        # creating a white mask where the object is located
        object_mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)
        object_mask[img[:,:,3]==255] = 255
        # measuring the pixel size of the object
        size_object = object_mask[object_mask==255].shape[0]

        # removing transparency in the background and make image 2D
        img[:,:,3] == 255

        # operate the occlusion for each level
        for level in ('low', 'high'):
            
            # create the write path, one for each level
            writePath = outputPathDeletion + "{}/".format(level)
            if not os.path.exists(writePath): os.makedirs(writePath)

            if level == 'low':
                nOccluders = nOccluders_low
                portionVisible = portionLow
            elif level == 'high':
                nOccluders = nOccluders_high
                portionVisible = portionHigh

            proportion_occluded = 0
                
            while abs(portionVisible - (1 - proportion_occluded)) > 1:
                
                # make a 2D copy of the image
                gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                
                # create the coordinates for occluders
                points_1 = np.array(randint(0, gray.shape[0], nOccluders))
                points_2 = np.array(randint(0, gray.shape[1], nOccluders))
                points = np.column_stack((points_1,points_2))

                # randomly generate the blob radiuses within the determined range
                radiuses = randint(sizeOccluder[0], sizeOccluder[1], nOccluders)

                # preparing the background for occluder mask
                # create the black figure that will hold the occluder mask
                occluder_mask = np.zeros((gray.shape[0], gray.shape[1]), np.uint8)

                # drawing occluders on the image & on their mask
                for point, radius in zip(points, radiuses):
                    cv.circle(gray, point, radius, tuple(colBckgrnd[0:3]), -1)
                    # here I draw the occluder mask
                    cv.circle(occluder_mask, point, radius, color = (255, 255, 255), thickness = -1)

                # calculate the intersection between masks
                intersection = cv.bitwise_and(object_mask, occluder_mask) # create the intersection
                intersectionSize = intersection[intersection==255].shape[0] # measure the size of intersection
                proportion_occluded = ((intersectionSize/size_object)*100)

                # write the image only if occlusion is as required
                if abs(portionVisible - (100 - proportion_occluded)) < 1:
                    cv.imwrite(writePath + 'deletion_{}_'.format(level) + name + '.png', gray)
                    print('Object occluded successfully')
                    break
                elif portionVisible > (100 - proportion_occluded):
                    #nOccluders += nOccluders
                    print('Occlusion is too low, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue
                elif portionVisible < (100 - proportion_occluded):
                    #nOccluders -= nOccluders
                    print('Occlusion is too high, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue

else:
    print('Deletion is off. Change parameters in the preamble to run it.')

Occlusion is too low, the difference is 51.469121746299706
Occlusion is too low, the difference is 46.16112441600251
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 13.520081661497386
Occlusion is too low, the difference is 66.64716736680931
Occlusion is too low, the difference is 58.657689136665226
Occlusion is too low, the difference is 63.51811864473322
Occlusion is too high, the difference is 29.787994189470382
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 23.586431628126107
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 68.24898904636645
Occlusion is too low, the difference is 69.67413921714892
Occlusion is too low, the difference is 61.15464646068078
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 55.08892465941659
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the di

Occlusion is too low, the difference is 5.077539162184451
Object occluded successfully
Occlusion is too low, the difference is 63.812103100485615
Occlusion is too low, the difference is 29.346283152782966
Occlusion is too low, the difference is 34.471423234964504
Occlusion is too high, the difference is 9.45461337317893
Occlusion is too low, the difference is 58.2816585730295
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 17.499066118789692
Occlusion is too low, the difference is 65.03175196115055
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 55.54538662682107
Occlusion is too low, the difference is 34.1539036234591
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 4.830033619723565
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 49.215539783339565
Oc

Occlusion is too high, the difference is 47.29610841108234
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 39.445528397601905
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 36.60044698286566
Occlusion is too high, the difference is 68.67678881833339
Occlusion is too high, the difference is 67.23296321260067
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 19.02621589981908
Object occluded successfully
Occlusion is too low, the difference is 28.428014036465356
Object occluded successfully
Occlusion is too low, the difference is 3.7797845451645458
Occlusion is too high, the difference is 63.71794557188781
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 57.54141002674781
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 8.953645309317565
Occlusion is too low, the difference is 28.939903315256075
Occlusio

Occlusion is too low, the difference is 11.74076531740765
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 14.990046449900468
Occlusion is too high, the difference is 31.90555186905552
Occlusion is too high, the difference is 5.888077858880777
Occlusion is too low, the difference is 11.306679938066793
Occlusion is too low, the difference is 22.13116567131165
Occlusion is too high, the difference is 32.7211900022119
Occlusion is too high, the difference is 35.837204158372046
Occlusion is too high, the difference is 27.76376907763769
Occlusion is too high, the difference is 10.132161026321612
Occlusion is too low, the difference is 19.949679274496802
Occlusion is too high, the difference is 9.949679274496795
Occlusion is too low, the difference is 29.784339747843404
Occlusion is too high, the difference is 19.198186241981865
Occlusion is too high, the difference is 1.3758018137580166
Occlusion is too low, the difference is 21.85744304357442
Occlusion 

Occlusion is too low, the difference is 29.0379386248867
Occlusion is too low, the difference is 19.410850705684318
Occlusion is too low, the difference is 54.90439811817515
Occlusion is too high, the difference is 18.514825844878928
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 24.26172903448574
Occlusion is too high, the difference is 13.126159955112428
Occlusion is too low, the difference is 2.193879753118395
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 5.762872804178002
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 26.70723811990159
Occlusion is too low, the difference is 68.8195433553455
Occlusion is too low, the difference is 38.75782295308386
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 66.2946178082783
Occlusion is too high, the difference is 25.273857309335753
Occlusion is to

...

Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 1.9537037037036953
Occlusion is too low, the difference is 20.77777777777778
Occlusion is too low, the difference is 66.89814814814814
Occlusion is too low, the difference is 53.98148148148148
Occlusion is too low, the difference is 66.86111111111111
Occlusion is too low, the difference is 66.85185185185186
Occlusion is too low, the difference is 9.731481481481481
Occlusion is too low, the difference is 30.185185185185183
Occlusion is too low, the difference is 51.074074074074076
Occlusion is too low, the difference is 60.30555555555554
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 46.953703703703695
Occlusion is too low, the difference is 68.29629629629629
Occlusion is too low, the difference is 34.14814814814814
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 32.12037037037037
Occlusion is too low, the difference is 20.88888

Occlusion is too high, the difference is 13.844924060751403
Occlusion is too high, the difference is 15.85931254996003
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 8.729016786570746
Occlusion is too low, the difference is 29.07274180655476
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 16.434852118305358
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 27.897681854516378
Occlusion is too low, the difference is 18.017585931255
Occlusion is too low, the difference is 15.147881694644283
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 29.096722621902472
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 29.91207034372502
Occlusion is too low, the difference is 1.071143085531

Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 31.010995723885152
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 41.91508857666463
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 14.777031154551004
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 58.973732437385465
Occlusion is too low, the difference is 63.929444105070246
Occlusion is too low, the difference is 64.67012828344534
Occlusion is too low, the difference is 68.65607819181429
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 20.630726939523527
Occlusion is too low, the difference is 41.724190592547345
Occlusion is too low, the difference is 11.49358582773366
Occlusion is too low, the difference is 32.102932193

Occlusion is too low, the difference is 4.4349419670128185
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 10.658216249236403
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 26.724190592547345
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 34.08827122785584
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 22.707697006719613
Occlusion is too high, the difference is 31.912034208918755
Occlusion is too low, the difference is 29.91600488698839
Occlusion is too low, the difference is 19.71441661576054
Occlusion is too low, the difference is 5.030543677458766
Object occluded successfully
Occlusion is too low, the difference is 41.99677506046763
Occlusion is too high, the difference is 30.0
Occlusion is too low, t

Occlusion is too low, the difference is 56.737832458838426
Occlusion is too low, the difference is 68.81038538085761
Occlusion is too high, the difference is 5.805138411434768
Occlusion is too low, the difference is 50.27410891984802
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 65.87931970327483
Occlusion is too low, the difference is 40.68481997466979
Occlusion is too low, the difference is 50.28767866835534
Occlusion is too high, the difference is 28.69730414329655
Occlusion is too low, the difference is 36.01682648814908
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 69.97286050298536
Occlusion is too low, the difference is 5.882938302876795
Occlusion is too low, the difference is 27.368373439478923
Occlusion is too low, the difference is 52.675954405645015
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 66.52162113262168
Oc

In [10]:
# partial viewing

if doManipulations[1]:
    for file in imagePath:
        
        # read the image
        img = cv.imread(file, -1)
        
        # extract image name
        basename = os.path.basename(file)
        name = os.path.splitext(basename)[0]
        
        # creating a mask where the object is located
        object_mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)
        object_mask[img[:,:,3]==255] = 255
        # measuring the pixel size of the object
        size_object = object_mask[object_mask==255].shape[0]

        # removing transparency in the background and make image 2D
        img[:,:,3] == 255

        # operate the occlusion for each level
        for level in ('low', 'high'):
            
            # create the write path, one for each level
            writePath = outputPathPartialViewing + "{}/".format(level)
            if not os.path.exists(writePath): os.makedirs(writePath)

            if level == 'low':
                nOccluders = nOccluders_low
                portionVisible = portionLow
            elif level == 'high':
                nOccluders = nOccluders_high
                portionVisible = portionHigh

            proportion_occluded = 0
                
            while abs(portionVisible - (1 - proportion_occluded)) > 1:
                
                # make a 2D copy of the image
                gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                
                # create the coordinates for occluders
                points_1 = np.array(randint(0, gray.shape[0], nOccluders))
                points_2 = np.array(randint(0, gray.shape[1], nOccluders))
                points = np.column_stack((points_1,points_2))

                # randomly generate the blob radiuses within the determined range
                radiuses = randint(sizeOccluder[0], sizeOccluder[1], nOccluders)
                
                # create the initial full mask
                occluder = np.zeros_like(gray)
                # setting the color of the mask
                occluder[:,:] = colPartialViewing

                # drawing occluders on the image & on their mask
                for point, radius in zip(points, radiuses):
                    cv.circle(occluder, point, radius, (255,255,255), -1)
                occluded = cv.bitwise_and(gray, gray, mask = occluder)
                
                # create the occluder mask
                occluder_inv = cv.bitwise_not(occluder)
                blank = np.ones_like(gray)
                blank[:] = 255
                occluder_mask = cv.bitwise_and(blank, blank, mask = occluder_inv)

                # measuring occluder size
                size_occluder = occluder_mask[occluder_mask==255].shape[0]

                # calculate the intersection between masks
                intersection = cv.bitwise_and(object_mask, occluder_mask) # create the intersection
                intersectionSize = intersection[intersection==255].shape[0] # measure the size of intersection
                proportion_occluded = ((intersectionSize/size_object)*100)

                # write the image only if occlusion is as required
                if abs(portionVisible - (100 - proportion_occluded)) < 1:
                    cv.imwrite(writePath + 'partialViewing_{}_'.format(level) + name + '.png', occluded)
                    print('Object occluded successfully')
                    break
                elif portionVisible > (100 - proportion_occluded):
                    #nOccluders += nOccluders
                    print('Occlusion is too low, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue
                elif portionVisible < (100 - proportion_occluded):
                    #nOccluders -= nOccluders
                    print('Occlusion is too high, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue

else:
    print('Deletion is off. Change parameters in the preamble to run it.')

Occlusion is too low, the difference is 13.606454399120572
Occlusion is too low, the difference is 23.90836637744887
Occlusion is too low, the difference is 6.916493266852498
Occlusion is too low, the difference is 6.0645439912056816
Occlusion is too low, the difference is 40.56652663813749
Occlusion is too low, the difference is 14.87456322876998
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 68.89285854501198
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 18.27293785088925
Occlusion is too low, the difference is 68.97923128263515
Occlusion is too low, the difference is 9.433080758509675
Occlusion is too low, the difference is 21.796160339209294
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 18.80452279062463
Occlusion is too low, the difference is 10.445997408817867
Object occluded successfully
Occlusion is too high, the d

Occlusion is too high, the difference is 68.19387373926037
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 32.89689951438177
Occlusion is too high, the difference is 26.143070601419495
Occlusion is too high, the difference is 43.61038475905865
Occlusion is too high, the difference is 45.521105715353
Occlusion is too high, the difference is 35.6966753828913
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 39.36122525214793
Occlusion is too high, the difference is 27.74000747104968
Occlusion is too high, the difference is 61.28128502054538
Occlusion is too high, the difference is 37.34590960029884
Occlusion is too high, the difference is 53.64774000747104
Occlusion is too low, the difference is 10.797534553604777
Occlusion is too high, the difference is 46.561449383638404
Occlusion is too high, the difference is 29.01755696675383
Occlusion is too low, the difference 

Occlusion is too high, the difference is 41.75848735322289
Occlusion is too low, the difference is 3.9721167831423543
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 63.447798786760785
Occlusion is too high, the difference is 60.90070594912909
Occlusion is too high, the difference is 5.198126928943921
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 50.82585405654689
Occlusion is too high, the difference is 68.37525275816807
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 43.54641881585016
Occlusion is too high, the difference is 15.893788357160595
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 52.00361843272198
Occlusion is too high, the difference is 32.64145588704814
Occlusion is too high, the difference is 64.67877540884743
Occlusion is too high, the difference is 68.99960977686331
O

Occlusion is too low, the difference is 3.9276630379489177
Occlusion is too low, the difference is 27.701699053820334
Occlusion is too low, the difference is 29.840777291687864
Occlusion is too low, the difference is 44.710041713297386
Occlusion is too low, the difference is 28.599552345101237
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 8.082714416522535
Occlusion is too low, the difference is 7.806490996032153
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 47.43412351205616
Occlusion is too low, the difference is 40.1419269508597
Occlusion is too high, the difference is 11.661410112931122
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 5.144979143351307
Occlusion is too low, the difference is 15.068165632312542
Occlusion is too lo

Occlusion is too high, the difference is 45.43493743005392
Occlusion is too high, the difference is 40.51836402482451
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 65.93041001119137
Occlusion is too high, the difference is 66.19747685420694
Occlusion is too high, the difference is 31.064197782073457
Occlusion is too high, the difference is 68.7307966222403
Occlusion is too high, the difference is 54.31173059314274
Occlusion is too high, the difference is 4.49740563638214
Occlusion is too high, the difference is 64.45772713399126
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 64.55437989622546
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 28.299420083426597
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 69.82704242547564
Occlusion is too high, the difference is 56.11506765693356
Occlusion is too high, the difference is 58.65093

...

Occlusion is too high, the difference is 57.11430855315747
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 28.705035971223026
Occlusion is too high, the difference is 54.22861710631494
Occlusion is too high, the difference is 11.910471622701841
Occlusion is too high, the difference is 70.0
Occlusion is too low, the difference is 5.907274180655477
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 44.412470023980816
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 14.972022382094323
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 22.80575539568345
Occlusion is too high, the difference is 39.92805755395683
Occlusion is too low, the difference is 25.011990407673864
Occlusion is too high, the difference i

Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 16.490807354116704
Occlusion is too high, the difference is 69.96802557953637
Occlusion is too high, the difference is 65.1558752997602
Occlusion is too low, the difference is 5.243804956035177
Occlusion is too high, the difference is 64.19664268585132
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 16.490807354116704
Occlusion is too high, the difference is 34.060751398880896
Occlusion is too high, the difference is 64.1726618705036
Occlusion is too high, the difference is 29.280575539568346
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 16.666666666666664
Occlusion is too high, the difference is 47.23421262989608
Occlusion is too high, the difference is 19.944044764188646
Occlusion is too low, the difference is 2.0223820943245414
Occlusion is too low, the difference is 15.70743405275779
Occlusion is too high, the differe

Occlusion is too high, the difference is 31.40687450039968
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 11.662669864108715
Occlusion is too high, the difference is 59.680255795363706
Occlusion is too high, the difference is 50.26378896882494
Occlusion is too high, the difference is 10.72741806554756
Occlusion is too high, the difference is 33.63709032773781
Occlusion is too high, the difference is 27.6898481215028
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 34.900079936051156
Occlusion is too high, the difference is 29.616306954436446
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 56.83453237410072
Occlusion is too high, the difference is 38.82494004796163
Occlusion is too high, the difference is 44.060751398880896
Occlusion is

Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 52.17012828344534
Occlusion is too high, the difference is 29.53420891875382
Occlusion is too high, the difference is 8.474343310934643
Occlusion is too high, the difference is 9.902260232131951
Occlusion is too low, the difference is 61.76084300549786
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 19.702199144777033
Occlusion is too low, the difference is 24.917532070861327
Occlusion is too low, the difference is 19.350946854001222
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 19.23335369578497
Occlusion is too high, the difference is 23.60109957238852
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 38.80726939523518
Occlusion is too high, the difference is 16.660048869883937
Occlusion is too high, the difference is 30.0
Occlusion is too l

Occlusion is too high, the difference is 23.568843857427176
Occlusion is too high, the difference is 70.0
Occlusion is too low, the difference is 8.053193414148723
Occlusion is too high, the difference is 35.962547494119775
Occlusion is too high, the difference is 44.045594354984615
Occlusion is too high, the difference is 69.74669802786322
Occlusion is too high, the difference is 48.73168083951511
Occlusion is too high, the difference is 67.01917857789036
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 67.79265424280803
Occlusion is too low, the difference is 29.51601230323864
Occlusion is too high, the difference is 69.26271033110186
Occlusion is too high, the difference is 62.89849828116519
Occlusion is too high, the difference is 63.87099692419034
Occlusion is too high, the difference is 54.44906821060249
Occlusion is too high, the difference is 15.698389723177137
Occlusion is too high, the difference is 46.985706531572276
Occlusion is too hig

In [11]:
# blobs

if doManipulations[2]:
    for file in imagePath:
        
        # read the image
        img = cv.imread(file, -1)
        
        # extract image name
        basename = os.path.basename(file)
        name = os.path.splitext(basename)[0]

        # creating object masks & measuring pixel size
        # creating a white mask where the object is located
        object_mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)
        object_mask[img[:,:,3]==255] = 255
        # measuring the pixel size of the object
        size_object = object_mask[object_mask==255].shape[0]

        # removing transparency in the background and make image 2D
        img[:,:,3] == 255

        # operate the occlusion for each level
        for level in ('low', 'high'):
            
            # create the write path, one for each level
            writePath = outputPathBlob + "{}/".format(level)
            if not os.path.exists(writePath): os.makedirs(writePath)

            if level == 'low':
                nOccluders = nOccluders_low
                portionVisible = portionLow
            elif level == 'high':
                nOccluders = nOccluders_high
                portionVisible = portionHigh

            proportion_occluded = 0
                
            while abs(portionVisible - (1 - proportion_occluded)) > 1:
                
                # make a 2D copy of the image
                gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                
                # create the coordinates for occluders
                points_1 = np.array(randint(0, gray.shape[0], nOccluders))
                points_2 = np.array(randint(0, gray.shape[1], nOccluders))
                points = np.column_stack((points_1,points_2))

                # randomly generate the blob radiuses within the determined range
                radiuses = randint(sizeOccluder[0], sizeOccluder[1], nOccluders)

                # preparing the background for occluder mask
                # create the black figure that will hold the occluder mask
                occluder_mask = np.zeros((gray.shape[0], gray.shape[1]), np.uint8)

                # drawing occluders on the image & on their mask
                for point, radius in zip(points, radiuses):
                    cv.circle(gray, point, radius, tuple(colBlob[0:3]), -1)
                    # here I draw the occluder mask
                    cv.circle(occluder_mask, point, radius, color = (255, 255, 255), thickness = -1)

                # calculate the intersection between masks
                intersection = cv.bitwise_and(object_mask, occluder_mask) # create the intersection
                intersectionSize = intersection[intersection==255].shape[0] # measure the size of intersection
                proportion_occluded = ((intersectionSize/size_object)*100)

                # write the image only if occlusion is as required
                if abs(portionVisible - (100 - proportion_occluded)) < 1:
                    cv.imwrite(writePath + 'blob_{}_'.format(level) + name + '.png', gray)
                    print('Object occluded successfully')
                    break
                elif portionVisible > (100 - proportion_occluded):
                    #nOccluders += nOccluders
                    print('Occlusion is too low, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue
                elif portionVisible < (100 - proportion_occluded):
                    #nOccluders -= nOccluders
                    print('Occlusion is too high, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue

else:
    print('Deletion is off. Change parameters in the preamble to run it.')

Occlusion is too low, the difference is 25.82427073927211
Occlusion is too low, the difference is 19.4405402222135
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 66.40375328805308
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 6.111656393545601
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 68.0487613364218
Occlusion is too low, the difference is 66.92591574732047
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 28.802559773860466
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 44.67708374229515
Occlusion is too low, the difference is 67.17718189313337
Occlusion is too low, the difference is 44.34729692591576
Occlusion is too low, the difference is 31.43457265125044
Occlusion is too low, the difference is 38.23053668878333
Occlusion is too low, the differen

Occlusion is too low, the difference is 21.468729142946884
Occlusion is too high, the difference is 68.35499195163126
Occlusion is too low, the difference is 27.506968709512776
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 27.530524910682743
Occlusion is too high, the difference is 61.888814730477804
Occlusion is too low, the difference is 23.840053394055985
Occlusion is too high, the difference is 35.282085509010244
Occlusion is too high, the difference is 30.170389855129365
Occlusion is too low, the difference is 19.164147461819326
Occlusion is too low, the difference is 6.302461623022253
Occlusion is too low, the difference is 25.269129598366774
Occlusion is too low, the difference is 1.1750618350280746
Occlusion is too low, the difference is 28.029131168780182
Occlusion is too high, the difference is 24.49334537316949
Occlusion is too high, the difference is 17.367594519257196
Occlusion is too low, 

Occlusion is too high, the difference is 6.457483415516691
Occlusion is too low, the difference is 12.482528645925711
Occlusion is too low, the difference is 6.820745680939368
Occlusion is too low, the difference is 1.3718116995991352
Occlusion is too high, the difference is 42.705665330448056
Occlusion is too low, the difference is 27.96374472311895
Occlusion is too low, the difference is 29.013799709106394
Occlusion is too low, the difference is 6.2460534250949
Occlusion is too high, the difference is 32.478271666252795
Occlusion is too low, the difference is 3.553513781971688
Occlusion is too low, the difference is 29.44659264252013
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 20.61194082798255
Occlusion is too high, the difference is 3.300223491432817
Occlusion is too high, the difference is 69.74458121962468
Occlusion is too high, the difference is 10.380999680726525
Occlusion is too low, the difference is 24.068608322395264
Occlusion is to

Occlusion is too high, the difference is 31.47788144311611
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 33.507041753875626
Occlusion is too high, the difference is 61.872716307779626
Occlusion is too low, the difference is 9.853843697896338
Occlusion is too low, the difference is 28.786760793217212
Occlusion is too low, the difference is 16.16126857994253
Occlusion is too low, the difference is 2.6914753982049717
Occlusion is too low, the difference is 26.597963744723117
Object occluded successfully
Occlusion is too low, the difference is 6.538980638511937
Occlusion is too low, the difference is 57.924222718460896
Occlusion is too low, the difference is 56.90829672891462
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 21.044146155922554
Occlusion is too low, the difference is 57.590488576967445
Occlusion is too low, the difference is 24.671051017152948
Occlusion is too low, the difference is 70.0
Occlusion is

Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 29.07162478380303
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 21.720927866517442
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 17.255570251297186
Occlusion is too low, the difference is 8.148845253840676
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 38.80150574829585
Occlusion is too high, the difference is 46.889815851053
Occlusion is too low, the difference is 21.436056567300852
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 14.841794689185065
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 19.280191270729482
Occlusion is too low, the difference is 30.0
Occlusion is too hig

...

Occlusion is too low, the difference is 51.21502797761791
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 29.04876099120704
Occlusion is too low, the difference is 67.01039168665068
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 8.161470823341332
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 47.28217426059153
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 20.551558752997607
Occlusion is too low, the difference is 65.36370903277378
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 32.150279776179055
Occlusion is too low, the difference is 6.834532374100718
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 26.22701838529177
Occlusion is

Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 4.172661870503603
Occlusion is too high, the difference is 63.24540367705835
Occlusion is too high, the difference is 30.271782573940847
Occlusion is too low, the difference is 30.0
Object occluded successfully
Occlusion is too low, the difference is 40.43812691648044
Occlusion is too low, the difference is 67.56769398679901
Occlusion is too low, the difference is 63.91923496699755
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 3.485785562080963
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 25.168650278052077
Occlusion is too low, the difference is 53.529962060183976
Occlusion is too high, the difference is 3.603243074684258
Occlusion is too high, the difference is 18.441349202224416
Occlusion is too low, the difference is 16.73873499298373
Occlusion is too low, the difference is 33

Occlusion is too low, the difference is 54.048564447159436
Occlusion is too low, the difference is 68.73243738546121
Occlusion is too low, the difference is 2.9948075748320093
Occlusion is too low, the difference is 26.597434331093467
Occlusion is too low, the difference is 53.651496640195475
Occlusion is too high, the difference is 25.34208918753818
Occlusion is too high, the difference is 15.667379352474043
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 55.56811240073304
Occlusion is too low, the difference is 7.469456322541234
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 68.4728161270617
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 1.2202199144776955
Occlusion is too low, the difference is 50.79566279780086
Occlusion is too low, the difference is 69.61820403176543
Occlusion is 

Object occluded successfully
Occlusion is too low, the difference is 32.57278509361283
Occlusion is too low, the difference is 61.44495207381527
Occlusion is too low, the difference is 34.57045597061722
Occlusion is too low, the difference is 6.450774881304312
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 58.73958613276001
Occlusion is too low, the difference is 9.075517334049984
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 66.37194302606827
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 55.219027143241064
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 55.56839559258263
Occlusion is too low, the difference is 63.49637194302606
Occlusion is too low, the difference is 69.8925020155872
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 70.0
Occlusion is too l

Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 23.388873958613274
Occlusion is too high, the difference is 5.617665502105169
Occlusion is too low, the difference is 2.104273044880401
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 43.31362536952432
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 38.485174236316396
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 7.165636477649386
Occlusion is too low, the difference is 30.0
Occlusion is too low, the difference is 30.0
Object occluded successfully
Occlusion is too low, the difference is 41.182377419938476
Occlusion is too low, the difference is 69.91858150895604
Occlusion is too low, the difference is 69.86430251492673
Occlusion is too low, the difference is 2.5628731680839536
Occlusion is too low, th

In [12]:
im = cv.imread(imagePath[1], -1)
print(im.shape)
np.unique(im[:,:,3])
plt.imshow(im)


(700, 700, 4)


NameError: name 'plt' is not defined